# Mistral OCR Workshop

This notebook demonstrates Optical Character Recognition using Mistral models on AWS.

## Introduction to Mistral OCR

**Mistral OCR** is a specialized optical character recognition model designed for extracting text, images, tables, and mathematical expressions from documents. Unlike traditional OCR models that only extract text, Mistral OCR comprehends each element of documents and returns ordered interleaved text and images in markdown format, making it ideal for multimodal document understanding and RAG systems.

### Key Features

- **State-of-the-Art Performance**: Achieves 94.89% overall accuracy on benchmarks, outperforming Google Document AI (83.42%), Azure OCR (89.52%), Gemini models (88-90%), and GPT-4o (89.77%)
- **Complex Document Understanding**: Excels at processing interleaved imagery, mathematical expressions, tables, and advanced layouts such as LaTeX formatting
- **Natively Multilingual**: Parses, understands, and transcribes thousands of scripts, fonts, and languages across all continents with 99%+ accuracy
- **Multi-format Support**: Process images (JPG, PNG, WebP, etc.) and multi-page PDF documents
- **Doc-as-Prompt & Structured Output**: Use documents as prompts and extract information into structured formats like JSON for downstream function calls and agent building
- **RAG-Ready**: Ideal model for use in Retrieval-Augmented Generation (RAG) systems with multimodal documents such as slides or complex PDFs
- **Production-Ready**: Deploy on Amazon SageMaker, la Plateforme, or self-host for organizations with stringent data privacy requirements

### Performance Highlights

Mistral OCR excels across multiple dimensions:

| Category | Mistral OCR 2503 | Best Competitor |
|----------|------------------|-----------------|
| Overall Accuracy | 94.89% | 90.23% (Gemini-1.5-Flash) |
| Mathematical Expressions | 94.29% | 89.11% (Gemini-1.5-Flash) |
| Scanned Documents | 98.96% | 96.15% (Gemini-1.5-Pro) |
| Tables | 96.12% | 91.70% (GPT-4o) |
| Multilingual | 99.02% | 97.31% (Azure OCR) |

### Use Cases

1. **Document Digitization**: Convert scanned documents, receipts, forms, and historical archives into searchable, AI-ready text
2. **Scientific Research**: Extract text, equations, tables, charts, and figures from scientific papers and journals
3. **Handwriting Recognition**: Process handwritten notes, whiteboard images, and forms
4. **Multimodal RAG Systems**: Build intelligent document understanding pipelines by combining Mistral OCR with LLMs like Mistral Small for analysis and summarization
5. **Multi-language Processing**: Handle documents in diverse linguistic backgrounds, from global organizations to hyperlocal businesses
6. **Structured Data Extraction**: Extract specific information from documents and format it into JSON for automated workflows and agent systems
7. **Cultural Heritage Preservation**: Digitize historical documents and artifacts for preservation and accessibility

This workshop will demonstrate how to deploy and use Mistral OCR on Amazon SageMaker for various document processing tasks, leveraging its industry-leading accuracy, speed, and versatility.



## Understanding Mistral OCR Response Format

Mistral OCR returns document content as **Markdown with interleaved text and images**, preserving the document structure and layout hierarchy. This format is optimized for downstream processing by LLMs and provides precise positioning information through bounding boxes.

### Supported Document Elements

Mistral OCR can extract and recognize a wide variety of document elements:

- **Standard typed text** - Regular printed text in any font
- **Multilingual text** - Mixed scripts (e.g., Asian and Roman characters)
- **Mathematical expressions** - Formulas, equations, and LaTeX notation
- **Handwriting** - Handwritten notes and annotations
- **Strikethrough text** - Text with strikethrough formatting
- **Diverse layout formats** - Complex page layouts and structures
- **Multi-column tables** - Tables with multiple columns and complex structures
- **Text with specific bounding boxes** - Precise spatial positioning
- **Text on colored backgrounds** - Text overlaid on colored regions
- **Form elements** - Checkboxes and circle selection fields

### Response Structure

Responses are returned in **Markdown format** with:
- Structural elements like pipes (`|`), LaTeX, and tables
- Layout cues that help LLMs understand document hierarchy
- Image placeholders embedded in the text

#### Image Representation

Images within documents are represented as Markdown image syntax:
```markdown
![img-0.jpeg](img-0.jpeg)
```

The image ID (e.g., `img-0.jpeg`) maps to the `pages[n].images` array, which contains:
- **Bounding box coordinates** (`top_left_x`, `top_left_y`, `bottom_right_x`, `bottom_right_y`)
- **Base64-encoded payload** (optional)
- **Image annotations** (if applicable)

#### Example Response Structure

```python
{
  'index': 13,
  'markdown': "![img-13.jpeg](img-13.jpeg)\n\nFigure 11: Examples of model responses...",
  'images': [
    {
      'id': 'img-13.jpeg',
      'top_left_x': 294,
      'top_left_y': 512,
      'bottom_right_x': 1404,
      'bottom_right_y': 1568,
      'image_base64': None,
      'image_annotation': None
    }
  ],
  'dimensions': {
    'dpi': 200,
    'height': 2200,
    'width': 1700
  }
}
```

This structured format enables precise extraction, spatial understanding, and seamless integration with downstream AI pipelines.

## Setup and Imports

First, let's import the necessary libraries for OCR processing.

In [ ]:
import base64
import os
import boto3
import json
from typing import Optional, Dict, Any
from IPython.display import Markdown, display

## Workshop Setup

This workshop demonstrates Mistral OCR using an endpoint deployed in your AWS account.

### Prerequisites

You should have:
1. **Deployed the Mistral OCR model** to a SageMaker endpoint in your AWS account
2. **Appropriate IAM permissions** to invoke the SageMaker endpoint
3. **AWS credentials configured** in your environment

### Endpoint Information

The endpoint ARN for this demo is:
```
arn:aws:sagemaker:us-west-2:314146324612:endpoint/mistral-ocr-endpoint
```

Let's configure the connection!

### Step 1: Configuration

Configure the endpoint details for your deployed Mistral OCR model.



In [ ]:
# Endpoint Configuration
# OCR Endpoint: us-west-2 (your deployed SageMaker endpoint)
# Bedrock (Pixtral Large): us-west-2 (configured separately in Bedrock cells)

ENDPOINT_NAME = "mistral-ocr-endpoint"  # Your deployed endpoint name
REGION = "us-west-2"  # AWS region where OCR endpoint is deployed

print("Endpoint Configuration:")
print(f"  OCR Endpoint: {ENDPOINT_NAME}")
print(f"  OCR Region: {REGION}")
print(f"  Bedrock Region: us-west-2 (for Pixtral Large)")
print("\n✅ Configuration ready!")

### Step 2: Create SageMaker Runtime Client

Now we'll create a SageMaker Runtime client to invoke your OCR endpoint.

In [ ]:
# Create SageMaker Runtime client using your AWS credentials
# This will use the credentials configured in your environment
sagemaker_client = boto3.client('sagemaker-runtime', region_name=REGION)

print("✅ SageMaker Runtime client created successfully!")
print(f"   Region: {REGION}")
print(f"   Ready to invoke endpoint: {ENDPOINT_NAME}")

# Store endpoint name for easy access throughout the notebook
MISTRAL_OCR_ENDPOINT_NAME = ENDPOINT_NAME

print("\n🎉 Setup complete! You're ready to use Mistral OCR.")

### Verification

Let's verify the client is properly configured.

In [ ]:
# Verify the client configuration
print("Client Configuration:")
print(f"  Service: sagemaker-runtime")
print(f"  Region: {sagemaker_client.meta.region_name}")
print(f"  Endpoint: {MISTRAL_OCR_ENDPOINT_NAME}")
print("\n✅ Client verification complete!")

## Helper Functions

These helper functions support image processing, model invocation, and post-response processing for the Mistral OCR model.

In [ ]:
def encode_local_file_base64(file_path: str, file_type: Optional[str] = None) -> str:
    """
    Encode a local file (image or PDF) to base64 string.
    
    Args:
        file_path: Path to the local file
        file_type: Type of file ('image' or 'pdf'). If None, inferred from extension.
    
    Returns:
        Base64 encoded string of the file
    """
    if file_type is None:
        ext = os.path.splitext(file_path)[1].lower()
        if ext == ".pdf":
            file_type = "pdf"
        elif ext in (".jpg", ".jpeg", ".png", ".gif", ".bmp", ".webp"):
            file_type = "image"
        else:
            raise ValueError(f"Unsupported file type from extension: {ext}")

    try:
        with open(file_path, "rb") as file:
            encoded_data = base64.b64encode(file.read()).decode("utf-8")
            return encoded_data
    except Exception as e:
        print(f"Failed to encode {file_type} at {file_path}: {e}")
        raise

def run_inference(client, endpoint_name: str, payload: dict[str, Any]) -> Dict[str, Any]:
    """
    Invoke the SageMaker endpoint for OCR inference.
    
    Args:
        client: SageMaker runtime client
        endpoint_name: Name of the deployed endpoint
        payload: JSON payload containing the image data
        
    Returns:
        Dictionary containing parsed OCR results
    """
    try:
        inference_out = client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="application/json",
            Body=json.dumps(payload)
        )
        inference_resp_str = inference_out["Body"].read().decode("utf-8")
        return json.loads(inference_resp_str)
    except Exception as e:
        print(f"Inference error: {e}")
        raise


def replace_images_in_markdown(markdown_str: str, images_dict: dict) -> str:
    """
    Replace image placeholders in markdown with base64-encoded images.
    
    Args:
        markdown_str: Markdown string with image placeholders
        images_dict: Dictionary mapping image names to base64 strings
    
    Returns:
        Markdown string with embedded base64 images
    """
    for img_name, base64_str in images_dict.items():
        markdown_str = markdown_str.replace(
            f"![{img_name}]({img_name})", f"![{img_name}]({base64_str})"
        )
    return markdown_str

def get_combined_markdown(ocr_response: dict) -> str:
    """
    Combine OCR text and images into a single markdown document.
    
    Args:
        ocr_response: Response dictionary from OCR model
    
    Returns:
        Combined markdown string with embedded images
    """
    markdowns = []
    for page in ocr_response["pages"]:
        image_data = {img["id"]: img["image_base64"] for img in page.get("images", [])}
        markdown_with_images = replace_images_in_markdown(page["markdown"], image_data)
        markdowns.append(markdown_with_images)
    return "\n\n".join(markdowns)

## Using Mistral OCR

Now let's use the Mistral OCR model to extract text from an image document. 

## Model Access and Deployment  

**Important Notice:**                                                                                                                                                              
The Mistral OCR model is available from AWS Marketplace via private offer. If you want to trial this model, please contact your account manager and provide your AWS account ID to whitelist for this model access.                                                                                                                                                   Then, please follow this link to deploy the Mistral OCR model on a SageMaker endpoint:                                                                                            [Mistral OCR SageMaker Deployment Guide](https://github.com/aws-samples/mistral-on-aws/blob/main/Mistral%20OCR/Mistral-OCR-SageMaker-Deployment-example.ipynb)     


**For This Demo:**                                                                                                                                                     
This notebook uses the Mistral OCR endpoint deployed in your AWS account (us-west-2 region). The endpoint is configured in the setup cells above.

# Part 1: Document Processing Examples

The following sections demonstrate Mistral OCR's capabilities across various document types. Each example showcases how the model handles different challenges in document understanding.

---

## Example 1: Whiteboard OCR

### Use Case
Extract text from whiteboard images captured during meetings, brainstorming sessions, or classroom lectures.

### Why This Matters
Whiteboards often contain:
- Handwritten text in various styles
- Diagrams and annotations
- Mixed content (text, arrows, shapes)
- Challenging lighting and perspective angles

### What This Code Does
1. **Encodes** the whiteboard image to base64 format
2. **Sends** the image to Mistral OCR endpoint with proper MIME type
3. **Extracts** all text content while preserving structure
4. **Displays** the results in readable markdown format

**Reference**: [Mistral OCR Documentation - Handwriting Recognition](https://docs.mistral.ai/capabilities/vision/#handwriting-recognition)


<img src="images/whiteboarding.jpg">

In [ ]:
# Process whiteboard image
whiteboard_b64 = encode_local_file_base64(file_path="images/whiteboarding.jpg")

# Prepare the payload for Mistral OCR model
whiteboard_payload = {
    "model": "mistral-ocr-2505",
    "document": {
        "type": "image_url",
        "image_url": f"data:image/png;base64,{whiteboard_b64}"
    }
}

# Invoke the endpoint
whiteboard_result = run_inference(
    client=sagemaker_client, 
    endpoint_name=MISTRAL_OCR_ENDPOINT_NAME, 
    payload=whiteboard_payload
)

# Display the extracted text
print("Extracted Whiteboard Content:")
display(Markdown(get_combined_markdown(whiteboard_result)))

## Example 2: Low-Resolution Handwriting OCR

### Use Case
Process handwritten notes from mobile phone photos, scanned documents, or low-quality images.

### Why This Matters
Real-world handwriting scenarios often involve:
- Low image resolution from quick phone captures
- Varying handwriting styles and legibility
- Compression artifacts from image processing
- Poor lighting conditions

### What This Code Does
1. **Loads** a low-resolution handwritten image
2. **Processes** the image despite quality limitations
3. **Extracts** handwritten text with high accuracy
4. **Returns** structured markdown output

This demonstrates Mistral OCR's robustness in handling challenging image quality while maintaining accuracy.

**Reference**: [Mistral OCR Documentation - Document Digitization](https://docs.mistral.ai/capabilities/vision/#document-digitization)

<img src="images/handwriting_low_res_2_resize.jpg">

In [ ]:
# Process low-resolution handwriting image
handwriting_b64 = encode_local_file_base64(file_path="images/handwriting_low_res_2_resize.jpg")

# Prepare the payload for Mistral OCR model
handwriting_payload = {
    "model": "mistral-ocr-2505",
    "document": {
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{handwriting_b64}"
    }
}

# Invoke the endpoint
handwriting_result = run_inference(
    client=sagemaker_client, 
    endpoint_name=MISTRAL_OCR_ENDPOINT_NAME, 
    payload=handwriting_payload
)

# Display the extracted text
print("Extracted Low-Resolution Handwriting Content:")
display(Markdown(get_combined_markdown(handwriting_result)))

## Example 3: Invoice Processing

### Use Case
Automate invoice data extraction for accounts payable, expense management, and financial auditing.

### Why This Matters
Invoice processing is critical for:
- **Accounts Payable Automation**: Reduce manual data entry by 90%+
- **Compliance & Auditing**: Maintain accurate financial records
- **Fraud Detection**: Identify duplicate or suspicious invoices
- **Payment Processing**: Extract vendor details, amounts, and due dates

### What This Code Does
1. **Encodes** the invoice image to base64
2. **Extracts** structured data including:
   - Vendor name and contact information
   - Invoice number and date
   - Line items with descriptions and amounts
   - Subtotals, taxes, and total amounts
3. **Preserves** table structures and formatting
4. **Outputs** in markdown format for easy downstream processing

The extracted data can be fed into accounting systems, RAG pipelines, or LLMs for further analysis.

**Reference**: [Mistral OCR Documentation - Structured Data Extraction](https://docs.mistral.ai/capabilities/vision/#structured-data-extraction)

<img src="images/invoice_1.jpg">

In [ ]:
# Process a single invoice
invoice_b64 = encode_local_file_base64(file_path="images/invoice_1.jpg")

# Prepare the payload for Mistral OCR model
invoice_payload = {
    "model": "mistral-ocr-2505",
    "document": {
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{invoice_b64}"
    }
}

# Invoke the endpoint
invoice_result = run_inference(
    client=sagemaker_client, 
    endpoint_name=MISTRAL_OCR_ENDPOINT_NAME, 
    payload=invoice_payload
)

# Display the extracted invoice content
print("Extracted Invoice Content:")
display(Markdown(get_combined_markdown(invoice_result)))

## Example 4: Official Document Processing

### Use Case
Extract information from official forms, legal documents, and government reports for insurance claims, legal proceedings, and compliance.

### Why This Matters
Official documents require:
- **High Accuracy**: Legal and compliance implications
- **Form Field Recognition**: Checkboxes, signatures, stamps
- **Mixed Content Handling**: Typed text, handwritten notes, and printed forms
- **Structured Extraction**: Preserve relationships between fields

### What This Code Does
1. **Processes** a police report with complex layout
2. **Extracts** both printed and handwritten content
3. **Identifies** form fields, checkboxes, and structured data
4. **Maintains** document hierarchy and relationships

### Real-World Applications
- **Insurance Claims**: Automatically process accident reports
- **Legal Discovery**: Extract evidence from official documents
- **Compliance Auditing**: Verify document completeness
- **Case Management**: Digitize paper-based records

**Reference**: [Mistral OCR Documentation - Form Processing](https://docs.mistral.ai/capabilities/vision/#form-elements)

<img src="images/national_youth_service_corps.jpg">

In [ ]:
# Process national service certificate image
national_service_cert_b64 = encode_local_file_base64(file_path="images/national_youth_service_corps.jpg")

# Prepare the payload for Mistral OCR model
national_service_cert_payload = {
    "model": "mistral-ocr-2505",
    "document": {
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{national_service_cert_b64}"
    }
}

# Invoke the endpoint
national_service_cert_result = run_inference(
    client=sagemaker_client, 
    endpoint_name=MISTRAL_OCR_ENDPOINT_NAME, 
    payload=national_service_cert_payload
)

# Display the extracted text
print("Extracted National Service Cert Content:")
display(Markdown(get_combined_markdown(national_service_cert_result)))

## Example 5: Mathematical Expression Recognition

### Use Case
Extract mathematical equations, formulas, and scientific notation from research papers, textbooks, and technical documents.

### Why This Matters
Mathematical content presents unique challenges:
- **Complex Notation**: Fractions, integrals, summations, matrices
- **LaTeX Formatting**: Convert visual math to machine-readable LaTeX
- **Subscripts & Superscripts**: Preserve mathematical meaning
- **Special Symbols**: Greek letters, operators, and mathematical symbols

### What This Code Does
1. **Recognizes** mathematical expressions in images
2. **Converts** visual formulas to LaTeX notation
3. **Preserves** mathematical structure and relationships
4. **Outputs** in markdown with embedded LaTeX for rendering

### Real-World Applications
- **Academic Research**: Digitize equations from scanned papers
- **Educational Content**: Convert textbook formulas to digital format
- **Scientific Publishing**: Extract equations for citation and reuse
- **STEM Documentation**: Build searchable mathematical knowledge bases

Mistral OCR achieves **94.29% accuracy** on mathematical expressions, outperforming competitors by 5%+.

**Reference**: [Mistral OCR Documentation - Mathematical Expressions](https://docs.mistral.ai/capabilities/vision/#mathematical-expressions)

<img src="images/math_formula.png">

In [ ]:
# Process mathematical formula image
math_formula_b64 = encode_local_file_base64(file_path="images/math_formula.png")

# Prepare the payload for Mistral OCR model
math_formula_payload = {
    "model": "mistral-ocr-2505",
    "document": {
        "type": "image_url",
        "image_url": f"data:image/png;base64,{math_formula_b64}"
    }
}

# Invoke the endpoint
math_formula_result = run_inference(
    client=sagemaker_client, 
    endpoint_name=MISTRAL_OCR_ENDPOINT_NAME, 
    payload=math_formula_payload
)

# Display the extracted mathematical content with LaTeX
print("Extracted Mathematical Formula (LaTeX):")
display(Markdown(get_combined_markdown(math_formula_result)))



# Part 2: Multimodal RAG: Combining OCR with LLMs

Now that we've seen how Mistral OCR extracts text from various document types, let's build an **intelligent document understanding pipeline** by combining OCR with Large Language Models (LLMs).

---

## What is Multimodal RAG?

**Retrieval-Augmented Generation (RAG)** enhances LLM responses by providing relevant context. In a multimodal RAG pipeline:

1. **OCR Model** (Mistral OCR) extracts structured text from documents
2. **LLM** (Pixtral Large) analyzes the extracted text to answer questions, summarize, or extract insights

This two-stage approach provides:
- **Higher Accuracy**: OCR specializes in text extraction, LLM specializes in understanding
- **Better Performance**: Smaller context windows for LLM processing
- **Cost Efficiency**: Process images once with OCR, reuse text for multiple queries
- **Flexibility**: Swap models independently based on requirements

## Pipeline Architecture

```
Document Image → Mistral OCR (us-west-2) → Structured Text → Pixtral Large (us-east-1) → Insights
```

**Important Note on Regions:**
- **OCR Endpoint**: us-west-2 (your deployed SageMaker endpoint)
- **Bedrock (Pixtral Large)**: us-east-1 (only region where Pixtral Large is available)
- These services can work together across regions without any issues

Let's set up the helper functions for invoking Bedrock models.

In [ ]:
# Helper functions for Bedrock Runtime
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-west-2")

def bedrock_converse(system_prompt: str, messages: list, endpoint_arn: str, display_usage=False):
    """Invoke model using Converse API"""
    system = [{"text": system_prompt}]
    
    response = bedrock_runtime.converse(
        modelId=endpoint_arn,
        messages=messages,
        system=system,
        additionalModelRequestFields={"max_tokens": 2000, "temperature": 0.3}
    )

    output_content = ''.join(
        content['text'] for content in response['output']['message']['content']
    )

    if display_usage:
        token_usage = response['usage']
        print(f"\tLatency: {response['metrics']['latencyMs']}ms")
    
    return output_content

def bedrock_converse_stream(system_prompt: str, messages: list, endpoint_arn: str):
    """Invoke model with streaming"""
    system = [{"text": system_prompt}]
    
    response = bedrock_runtime.converse_stream(
        modelId=endpoint_arn,
        messages=messages,
        system=system,
        additionalModelRequestFields={"max_tokens": 2000, "temperature": 0.3}
    )
    
    stream = response.get('stream')
    output_content = ''
    
    if stream:
        for event in stream:
            if 'messageStart' in event:
                print(f"\nRole: {event['messageStart']['role']}")
            
            if 'contentBlockDelta' in event:
                text_chunk = event['contentBlockDelta']['delta']['text']
                print(text_chunk, end="")
                output_content += text_chunk
            
            if 'messageStop' in event:
                print(f"\nStop reason: {event['messageStop']['stopReason']}")
            
            if 'metadata' in event:
                metadata = event['metadata']
                if 'metrics' in metadata:
                    print(f"Latency: {metadata['metrics']['latencyMs']}ms")
    
    return output_content

## Document Understanding Pipeline

This function orchestrates the complete document understanding workflow:

### Pipeline Steps

1. **Image Encoding**: Convert local document image to base64
2. **OCR Extraction**: Send to Mistral OCR endpoint to extract structured text
3. **Text Processing**: Convert OCR output to clean markdown format
4. **LLM Analysis**: Send extracted text + user query to Pixtral Large on Bedrock
5. **Stream Response**: Display insights in real-time

### Why This Approach?

- **Separation of Concerns**: OCR handles extraction, LLM handles reasoning
- **Reusability**: Extract once, query multiple times
- **Transparency**: See both raw OCR output and LLM analysis
- **Flexibility**: Easily swap models or add processing steps

Let's define the pipeline function:

In [ ]:
def document_understanding_pipeline(
    image_path: str,
    user_prompt: str,
    ocr_endpoint: str=None,
    llm_endpoint: str=None
) -> str:
    """
    Full pipeline for document understanding from image input.

    Args:
        image_path: Local path to the document image
        user_prompt: What insights the user wants to extract
        ocr_endpoint: SageMaker endpoint for OCR model  
        llm_endpoint: Bedrock inference profile for Pixtral Large (e.g., "us.mistral.pixtral-large-2502-v1:0")

    Returns:
        Model-generated response with document insights
    """

    # Step 1: Encode local file using helper
    encoded_image = encode_local_file_base64(image_path)

    # Prepare OCR payload
    api_payload = {
        "model": "mistral-ocr-2505",
        "document": {
            "type": "image_url",
            "image_url": f"data:image/jpeg;base64,{encoded_image}",  # Fixed: was image_b64
        },
        "include_image_base64": False
    }

    # Step 2: Run OCR model
    print("Step 1/2: Running Mistral OCR...")
    ocr_result = run_inference(client=sagemaker_client, endpoint_name=MISTRAL_OCR_ENDPOINT_NAME, payload=api_payload)

    # Step 3: Convert OCR output to Markdown
    print("Extracting text from document...")
    markdown_doc = get_combined_markdown(ocr_result)

    print("\n" + "="*80)
    print("📄 EXTRACTED TEXT FROM OCR")
    print("="*80)
    display(Markdown(markdown_doc))
    print("="*80 + "\n")

    # Step 4: Prepare LLM messages for Bedrock
    system_prompt = (
        "You are a document understanding assistant powered by Mistral AI. "
        "The user will provide structured OCR content from a scanned document. "
        "Use that information to generate clear, factual insights that answer the user's request. "
        "Be thorough and reference specific details from the document."
    )

    messages = [
        {
            "role": "user",
            "content": [{"text": f"{user_prompt}\n\n--- Document Content ---\n{markdown_doc}"}]
        }
    ]

    # Step 5: Call Bedrock Pixtral Large with streaming
    print("Step 2/2: Running Pixtral Large on Bedrock for analysis...")
    print("-"*80)
    insights = bedrock_converse_stream(system_prompt, messages, llm_endpoint)
    
    print("\n" + "="*80)
    print("✅ Analysis complete!")
    
    return insights

## Example: Document Summarization

Let's use the pipeline to extract and summarize a document.

<img src="images/french.png">

In [ ]:
image_path = "images/french.png"
user_prompt = "can you summarise this document"

# Using Pixtral Large 2502 - multimodal model with vision capabilities
llm_endpoint = "us.mistral.pixtral-large-2502-v1:0"

# Alternative options:
# llm_endpoint = "mistral.mistral-large-2407-v1:0"  # Mistral Large 2 (text-only)
# llm_endpoint = "mistral.mistral-small-2402-v1:0"  # Mistral Small (faster, cheaper)

document_understanding_pipeline(image_path=image_path, user_prompt=user_prompt, llm_endpoint=llm_endpoint)


# Part 3: Advanced: Multi-Agent Document Intelligence with OCR + Strands Agents

In this section, we'll go beyond simple OCR extraction to build a **sophisticated multi-agent document processing system** using **Strands Agents**. This demonstrates how to orchestrate multiple specialized AI agents that work together to analyze documents, enforce compliance rules, and generate actionable insights.

---

### What is Strands Agents?

**Strands Agents** is a Python framework for building agentic AI applications with tool use capabilities. It provides:
- **Agent Orchestration**: Coordinate multiple specialized agents in workflows
- **Tool Functions**: Decorate Python functions with `@tool` to make them callable by agents
- **Model Integration**: Works seamlessly with Amazon Bedrock models like Pixtral Large
- **State Management**: Track execution across multi-step workflows

Learn more: [https://github.com/strands-agents/strands-agents](https://github.com/strands-agents/strands-agents)

### What We'll Build: Invoice & Document Compliance Workflow

We'll create an **intelligent document processing pipeline** that:
1. **Extracts** text from invoices and official documents using Mistral OCR
2. **Classifies** document types automatically (invoice, report, form, etc.)
3. **Analyzes** documents with domain-specific agents
4. **Validates** against compliance rules (duplicate detection, anomaly detection)
5. **Synthesizes** findings into an executive summary with flagged issues

### Real-World Use Cases

This pattern is applicable to:
- **Accounts Payable Automation**: Process invoices, detect duplicates, flag anomalies
- **Insurance Claims Processing**: Analyze claim documents and police reports
- **Legal Document Review**: Extract key information from contracts and forms
- **Compliance Auditing**: Validate documents against regulatory requirements
- **Research Paper Analysis**: Extract equations, citations, and key findings

### Multi-Agent Architecture

Our workflow uses 5 specialized agents:

1. **Document Triage Agent**: Classifies document type (invoice, police report, form)
2. **Invoice Analysis Agent**: Extracts vendor, amounts, line items, checks for issues
3. **Report Analysis Agent**: Analyzes official documents like police reports
4. **Compliance Agent**: Validates documents against business rules
5. **Synthesis Agent**: Generates executive summary with prioritized actions

### Why Multi-Agent vs Single-Agent?

| Single Agent | Multi-Agent Workflow |
|-------------|---------------------|
| Generic prompts | Specialized expertise per domain |
| Limited context window | Distributed processing |
| Hard to maintain | Modular and scalable |
| One-size-fits-all | Tailored analysis per document type |

### Workflow Steps

```
1. OCR Extraction → 2. Document Triage → 3. Specialized Analysis → 4. Compliance Check → 5. Synthesis
```

Let's build it!

### Setup: Install and Import Strands Agents

First, let's install Strands Agents and import the necessary components.

In [ ]:
# Install Strands Agents (if not already installed)
# !pip install strands-agents>=0.1.6

# Import Strands components
from strands import Agent
from strands.models import BedrockModel
from strands.tools import tool
from dataclasses import dataclass, field
from typing import List
from datetime import datetime

print("✅ Strands Agents imported successfully!")

### Step 1: Define Document Inventory

First, we'll set up our document inventory and caching system.

#### What This Code Does

- **Document Registry**: Creates a dictionary mapping document IDs to file paths
- **Lazy Loading**: Documents are only processed when needed by agents
- **OCR Caching**: Stores OCR results to avoid redundant API calls
- **Type Classification**: Initially marked as "unknown" - will be classified by the triage agent

#### Why Caching Matters

In a multi-agent workflow, multiple agents may need to access the same document. Caching ensures:
- **Cost Efficiency**: OCR is called only once per document
- **Performance**: Subsequent accesses are instant
- **Consistency**: All agents work with identical extracted text

In [ ]:
# Document metadata (file paths for processing)
# The OCR text will be extracted dynamically when needed

DOCUMENTS_TO_PROCESS = {
    "doc_001": {
        "doc_id": "doc_001",
        "file_path": "images/invoice_1.jpg",
        "doc_type": "unknown"  # Will be classified by triage agent
    },
    "doc_002": {
        "doc_id": "doc_002", 
        "file_path": "images/invoice_2.jpg",
        "doc_type": "unknown"
    },
    "doc_003": {
        "doc_id": "doc_003",
        "file_path": "images/Invoice_3.jpg",
        "doc_type": "unknown"
    },
    "doc_004": {
        "doc_id": "doc_004",
        "file_path": "images/national_youth_service_corps.jpg",
        "doc_type": "unknown"
    }
}

# Cache for extracted OCR results (to avoid re-processing same images)
OCR_CACHE = {}

print(f"✅ Document metadata created for {len(DOCUMENTS_TO_PROCESS)} documents")
print("\nDocuments to process:")
for doc_id, doc in DOCUMENTS_TO_PROCESS.items():
    print(f"  - {doc_id}: {doc['file_path']}")

### Step 2: Create Agent Tools

Now we'll define the **tools** that agents can use to interact with documents and perform analysis.

#### What are Agent Tools?

In Strands Agents, tools are Python functions decorated with `@tool` that agents can call during their reasoning process. Think of them as the "hands" of the agent - they enable agents to:
- Access external data (OCR extraction)
- Query databases (document listing)
- Perform computations (duplicate detection)

#### Tools We're Creating

1. **`extract_document_text(doc_id)`**
   - Calls Mistral OCR to extract text from a document
   - Implements caching to avoid redundant API calls
   - Returns structured JSON with document metadata and OCR text

2. **`list_documents(doc_type)`**
   - Lists all available documents, optionally filtered by type
   - Helps agents discover what documents need processing

3. **`check_duplicate_invoice(doc_id, vendor, amount, date)`**
   - Compares invoice details against previously processed documents
   - Detects potential duplicate submissions
   - Returns match results with confidence indicators

#### How Agents Use Tools

When an agent needs information, it:
1. Decides which tool to call based on its task
2. Generates the appropriate parameters
3. Receives the tool output
4. Incorporates the results into its reasoning

This is similar to how ChatGPT uses function calling, but orchestrated within a multi-agent workflow.

In [ ]:
@tool
def extract_document_text(doc_id: str) -> str:
    """
    Extract text from a document using Mistral OCR.
    Caches results to avoid re-processing.
    
    Args:
        doc_id: Document ID to extract text from
    
    Returns:
        JSON string with document details and extracted OCR text
    """
    if doc_id not in DOCUMENTS_TO_PROCESS:
        return json.dumps({"error": f"Document {doc_id} not found"})
    
    # Check cache first
    if doc_id in OCR_CACHE:
        print(f"  [Using cached OCR for {doc_id}]")
        return json.dumps(OCR_CACHE[doc_id], indent=2)
    
    # Extract from document
    doc_info = DOCUMENTS_TO_PROCESS[doc_id]
    file_path = doc_info['file_path']
    
    print(f"  [Running OCR on {file_path}...]")
    
    try:
        # Encode image
        image_b64 = encode_local_file_base64(file_path=file_path)
        
        # Prepare OCR payload
        ocr_payload = {
            "model": "mistral-ocr-2505",
            "document": {
                "type": "image_url",
                "image_url": f"data:image/jpeg;base64,{image_b64}"
            },
            "include_image_base64": False
        }
        
        # Call OCR endpoint
        ocr_result = run_inference(client=sagemaker_client, endpoint_name=MISTRAL_OCR_ENDPOINT_NAME, payload=ocr_payload)
        
        # Extract markdown text
        ocr_text = get_combined_markdown(ocr_result)
        
        # Cache result
        result = {
            "doc_id": doc_id,
            "file_path": file_path,
            "ocr_text": ocr_text,
            "doc_type": doc_info['doc_type']
        }
        OCR_CACHE[doc_id] = result
        
        print(f"  [OCR complete: {len(ocr_text)} characters extracted]")
        return json.dumps(result, indent=2)
        
    except Exception as e:
        return json.dumps({"error": f"OCR extraction failed: {str(e)}"})


@tool
def list_documents(doc_type: str = None) -> str:
    """
    List all documents available for processing.
    
    Args:
        doc_type: Filter by document type (optional)
    
    Returns:
        JSON string with list of documents
    """
    docs = list(DOCUMENTS_TO_PROCESS.values())
    
    if doc_type and doc_type != "unknown":
        docs = [d for d in docs if d['doc_type'] == doc_type]
    
    result = [{"doc_id": d['doc_id'], "file_path": d['file_path'], "doc_type": d['doc_type']}
              for d in docs]
    
    return json.dumps(result, indent=2)


@tool
def check_duplicate_invoice(doc_id: str, vendor: str, amount: float, date: str) -> str:
    """
    Check if an invoice might be a duplicate by comparing with other processed invoices.
    
    Args:
        doc_id: Current document ID (to exclude from comparison)
        vendor: Vendor name
        amount: Invoice amount
        date: Invoice date
    
    Returns:
        JSON string with duplicate check results
    """
    matches = []
    
    # Check all cached documents
    for cached_doc_id, cached_doc in OCR_CACHE.items():
        if cached_doc_id == doc_id:
            continue  # Skip self
            
        # Simple heuristic: check if vendor and amount appear in OCR text
        ocr_text = cached_doc.get('ocr_text', '').lower()
        
        if (vendor.lower() in ocr_text and 
            str(amount) in ocr_text and
            date in ocr_text):
            matches.append({
                "doc_id": cached_doc_id,
                "file_path": cached_doc.get('file_path'),
                "match_reason": "Vendor, amount, and date found in document"
            })
    
    result = {
        "is_duplicate": len(matches) > 0,
        "match_count": len(matches),
        "matches": matches,
        "note": "This is a heuristic check based on OCR text content"
    }
    
    return json.dumps(result, indent=2)


print("✅ Tool functions defined:")
print("  - extract_document_text(doc_id) - Calls OCR API dynamically")
print("  - list_documents(doc_type)")
print("  - check_duplicate_invoice(doc_id, vendor, amount, date)")

### Step 3: Define Specialized Agents

Now we'll create our team of specialized AI agents, each with domain expertise and specific tools.

#### Agent Architecture

Each agent is configured with:
- **System Prompt**: Defines the agent's role, expertise, and instructions
- **Tools**: Specific functions the agent can call
- **Model**: Pixtral Large on Bedrock for reasoning and tool orchestration

#### Our Agent Team

**1. Triage Agent** 🏷️
- **Role**: Document classification specialist
- **Tools**: `extract_document_text`
- **Task**: Classify documents as invoice, police_report, contract, form, or other

**2. Invoice Analyst Agent** 💰
- **Role**: Invoice processing expert
- **Tools**: `extract_document_text`, `check_duplicate_invoice`
- **Task**: Extract vendor details, amounts, dates; validate structure; detect anomalies and duplicates

**3. Report Analyst Agent** 📋
- **Role**: Official document specialist
- **Tools**: `extract_document_text`
- **Task**: Extract incident details, parties involved, financial impacts from reports

**4. Compliance Agent** 🛡️
- **Role**: Compliance validation specialist
- **Tools**: `list_documents`, `extract_document_text`, `check_duplicate_invoice`
- **Task**: Check for duplicates, missing fields, threshold violations; assign risk levels

**5. Synthesis Agent** 📊
- **Role**: Executive summary specialist
- **Tools**: None (works with aggregated data from other agents)
- **Task**: Create executive summary with prioritized findings and recommended actions

#### Why Specialized Agents?

- **Domain Expertise**: Each agent has tailored prompts for specific document types
- **Tool Access Control**: Agents only have tools relevant to their role
- **Scalability**: Easy to add new agent types (e.g., contract analyst, receipt processor)
- **Maintainability**: Update one agent without affecting others

In [ ]:
# Initialize Bedrock model for agents
# CRITICAL: us.* model IDs are ONLY available in US regions (us-east-1, us-west-2)
# Your AWS profile is set to ap-southeast-1, so we need to explicitly override it
BEDROCK_REGION = "us-west-2"  # Must be US region for us.* models

# Create explicit boto3 session to override your default AWS profile region
import boto3
bedrock_boto_session = boto3.Session(region_name=BEDROCK_REGION)

# Initialize BedrockModel with explicit session to force correct region
bedrock_model_agents = BedrockModel(
    model_id="us.mistral.pixtral-large-2502-v1:0",
    region=BEDROCK_REGION,  # Explicitly set to us-west-2
    streaming=False,
    boto_session=bedrock_boto_session  # This overrides your AWS profile!
)

print(f"✅ Bedrock model configured for agents:")
print(f"   Model: us.mistral.pixtral-large-2502-v1:0")
print(f"   Bedrock Region: {BEDROCK_REGION}")
print(f"   Boto Session Region: {bedrock_boto_session.region_name}")
print(f"   OCR Endpoint Region: {REGION} (us-west-2)")
print(f"   ✓ Overriding your AWS profile default (ap-southeast-1)\n")

def create_triage_agent():
    """Agent that classifies document types"""
    system_prompt = """You are a document classification specialist. 
Analyze document content and classify it as: invoice, police_report, contract, form, or other.
Provide brief reasoning for your classification."""
    
    agent = Agent(model=bedrock_model_agents, tools=[extract_document_text], system_prompt=system_prompt)
    return agent

def create_invoice_analyst_agent():
    """Agent specialized in invoice analysis"""
    system_prompt = """You are an invoice analysis specialist. Your role:
1. Extract key information (vendor, invoice number, amounts, dates)
2. Validate invoice structure and completeness
3. Check for anomalies (unusual amounts, missing fields, calculation errors)
4. Use check_duplicate_invoice to detect potential duplicates
5. Flag potential issues for review

Be thorough and flag anything suspicious."""
    
    agent = Agent(model=bedrock_model_agents, 
                 tools=[extract_document_text, check_duplicate_invoice], 
                 system_prompt=system_prompt)
    return agent

def create_report_analyst_agent():
    """Agent specialized in official report analysis"""
    system_prompt = """You are an official document analyst specializing in reports.
Extract key information such as:
- Report numbers and dates
- Incident types and locations  
- Key parties involved
- Financial impacts
- Status and follow-up actions

Provide structured summaries."""
    
    agent = Agent(model=bedrock_model_agents, tools=[extract_document_text], system_prompt=system_prompt)
    return agent

def create_compliance_agent():
    """Agent that validates compliance rules"""
    system_prompt = """You are a compliance validation specialist. Check documents for:
1. Duplicate submissions (use check_duplicate_invoice tool)
2. Missing required fields
3. Amounts exceeding thresholds ($5,000+)
4. Date inconsistencies
5. Policy violations

Assign risk levels: LOW, MEDIUM, HIGH, CRITICAL."""
    
    agent = Agent(model=bedrock_model_agents, 
                 tools=[list_documents, extract_document_text, check_duplicate_invoice], 
                 system_prompt=system_prompt)
    return agent

def create_synthesis_agent():
    """Agent that synthesizes findings"""
    system_prompt = """You are an executive synthesis specialist. Create:
1. Executive Summary
2. Key Findings (prioritized by risk)
3. Flagged Issues requiring immediate attention
4. Recommended Actions

Be concise and actionable."""
    
    agent = Agent(model=bedrock_model_agents, tools=[], system_prompt=system_prompt)
    return agent

print("✅ Specialized agents created:")
print("  - Triage Agent")
print("  - Invoice Analyst Agent")
print("  - Report Analyst Agent")
print("  - Compliance Agent")
print("  - Synthesis Agent")

### Workflow State Management

Define the state object to track the multi-agent workflow.

In [ ]:
@dataclass
class DocumentWorkflowState:
    """State object for document processing workflow"""
    document_ids: List[str]
    
    # Classification results
    document_classifications: Dict[str, str] = field(default_factory=dict)
    
    # Analysis results
    invoice_analysis: Dict[str, Any] = field(default_factory=dict)
    report_analysis: Dict[str, Any] = field(default_factory=dict)
    
    # Compliance results
    compliance_findings: List[Dict] = field(default_factory=list)
    flagged_issues: List[Dict] = field(default_factory=list)
    
    # Final output
    executive_summary: str = ""
    
    # Tracking
    agents_invoked: List[str] = field(default_factory=list)
    execution_time: float = 0.0

print("✅ Workflow state management ready!")

### Step 4: Orchestrate the Multi-Agent Workflow

Now we'll create the orchestrator that coordinates all agents in a sequential workflow.

#### Workflow Execution Flow

```
┌─────────────────────────────────────────────────────────────────┐
│  STEP 1: TRIAGE & CLASSIFICATION                                │
│  • Triage Agent classifies each document                        │
│  • Calls extract_document_text for each doc                     │
│  • Updates document_classifications in state                    │
└─────────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 2: SPECIALIZED ANALYSIS                                   │
│  • Route invoices → Invoice Analyst Agent                       │
│  • Route reports → Report Analyst Agent                         │
│  • Each agent performs domain-specific extraction               │
│  • Results stored in state.invoice_analysis / report_analysis   │
└─────────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 3: COMPLIANCE VALIDATION                                  │
│  • Compliance Agent reviews all documents                       │
│  • Checks for duplicates, anomalies, threshold violations       │
│  • Assigns risk levels (LOW, MEDIUM, HIGH, CRITICAL)            │
│  • Results stored in state.compliance_findings                  │
└─────────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────────┐
│  STEP 4: EXECUTIVE SYNTHESIS                                    │
│  • Synthesis Agent aggregates all findings                      │
│  • Creates executive summary with prioritized actions           │
│  • Generates final report                                       │
└─────────────────────────────────────────────────────────────────┘
```

#### What This Orchestrator Does

1. **State Management**: Tracks all intermediate results in `DocumentWorkflowState`
2. **Sequential Execution**: Each step builds on previous results
3. **Agent Coordination**: Routes documents to appropriate specialist agents
4. **Progress Tracking**: Displays real-time progress with emojis and status updates
5. **Performance Metrics**: Measures execution time and agent invocations

#### Real-World Benefits

- **Auditability**: Complete trace of which agents processed which documents
- **Debugging**: State object shows intermediate results at each step
- **Scalability**: Easy to add new workflow steps or parallel processing
- **Flexibility**: Modify agent behavior without changing orchestration logic

In [ ]:
def run_document_workflow(document_ids: List[str]) -> DocumentWorkflowState:
    """
    Orchestrate multi-agent document processing workflow.
    
    Workflow Steps:
    1. Triage - Classify each document (calls OCR)
    2. Specialized Analysis - Route to appropriate analyst
    3. Compliance Check - Validate against rules
    4. Synthesis - Generate executive summary
    
    Args:
        document_ids: List of document IDs to process
        
    Returns:
        DocumentWorkflowState with complete analysis
    """
    import time
    start_time = time.time()
    
    state = DocumentWorkflowState(document_ids=document_ids)
    
    print("="*80)
    print("📄 DOCUMENT PROCESSING WORKFLOW")
    print("="*80)
    print(f"Processing {len(document_ids)} documents\n")
    
    # STEP 1: Triage and Classification
    print("\n📋 STEP 1: Document Triage & Classification")
    print("-"*80)
    triage_agent = create_triage_agent()
    state.agents_invoked.append("triage")
    
    for doc_id in document_ids:
        query = f"Classify the document type for document {doc_id}. Use extract_document_text to retrieve and analyze it."
        response = triage_agent(query)
        response_str = str(response)
        
        # Determine classification
        if "invoice" in response_str.lower():
            doc_type = "invoice"
        elif "police" in response_str.lower() or "report" in response_str.lower():
            doc_type = "police_report"
        else:
            doc_type = "other"
        
        state.document_classifications[doc_id] = doc_type
        print(f"  ✓ {doc_id}: {doc_type}")
    
    # STEP 2: Specialized Analysis
    print("\n\n🔍 STEP 2: Specialized Analysis")
    print("-"*80)
    
    invoice_agent = create_invoice_analyst_agent()
    report_agent = create_report_analyst_agent()
    
    for doc_id, doc_type in state.document_classifications.items():
        if doc_type == "invoice":
            print(f"\n  Analyzing invoice {doc_id}...")
            query = f"Analyze invoice {doc_id}. Extract all details, check for anomalies, and use check_duplicate_invoice to detect duplicates."
            response = invoice_agent(query)
            state.invoice_analysis[doc_id] = str(response)
            state.agents_invoked.append(f"invoice_analyst_{doc_id}")
            print(f"  ✓ Invoice analysis complete")
            
        elif doc_type == "police_report":
            print(f"\n  Analyzing police report {doc_id}...")
            query = f"Analyze police report {doc_id}. Extract key incident details."
            response = report_agent(query)
            state.report_analysis[doc_id] = str(response)
            state.agents_invoked.append(f"report_analyst_{doc_id}")
            print(f"  ✓ Report analysis complete")
    
    # STEP 3: Compliance Check
    print("\n\n🛡️  STEP 3: Compliance Validation")
    print("-"*80)
    compliance_agent = create_compliance_agent()
    state.agents_invoked.append("compliance")
    
    query = "Review all processed invoices for compliance issues: duplicates, missing fields, amounts >$5000, anomalies. List all documents first."
    response = compliance_agent(query)
    state.compliance_findings = [str(response)]
    print(f"  ✓ Compliance check complete")
    
    # STEP 4: Synthesis
    print("\n\n💡 STEP 4: Executive Synthesis")
    print("-"*80)
    synthesis_agent = create_synthesis_agent()
    state.agents_invoked.append("synthesis")
    
    synthesis_context = f"""Based on document processing results:

DOCUMENTS PROCESSED: {len(document_ids)}
CLASSIFICATIONS: {state.document_classifications}

INVOICE ANALYSIS:
{state.invoice_analysis}

REPORT ANALYSIS:
{state.report_analysis}

COMPLIANCE FINDINGS:
{state.compliance_findings}

Create an executive summary with key findings and recommended actions."""
    
    response = synthesis_agent(synthesis_context)
    state.executive_summary = str(response)
    
    state.execution_time = time.time() - start_time
    
    # Display final results
    print("\n\n" + "="*80)
    print("📝 EXECUTIVE SUMMARY")
    print("="*80)
    print(state.executive_summary)
    
    print("\n" + "="*80)
    print("📊 WORKFLOW SUMMARY")
    print("="*80)
    print(f"Documents Processed: {len(document_ids)}")
    print(f"Agents Invoked: {len(state.agents_invoked)}")
    print(f"Total Execution Time: {state.execution_time:.2f}s")
    print(f"OCR Cache Entries: {len(OCR_CACHE)}")
    print("="*80)
    
    return state

print("✅ Workflow orchestrator ready!")

### Step 5: Execute the Workflow

Now let's run the complete multi-agent workflow on our document batch.

#### What Will Happen

When you run this cell, you'll see:

1. **Real-time Progress Updates** 📊
   - Each workflow step displays with visual separators
   - OCR extraction progress for each document
   - Agent invocations and classifications

2. **Document Processing** 🔄
   - 3 invoices will be routed to the Invoice Analyst Agent
   - 1 police report will be routed to the Report Analyst Agent
   - Each document is processed only once (cached)

3. **Compliance Analysis** 🛡️
   - Duplicate detection across all invoices
   - Threshold checks (amounts > $5,000)
   - Risk level assignments

4. **Executive Summary** 📋
   - Consolidated findings from all agents
   - Prioritized action items
   - Risk assessment

#### Expected Output

- **Processing Time**: ~30-60 seconds (depending on document complexity)
- **Agent Invocations**: 9-10 total (1 triage + 3-4 analysts + 1 compliance + 1 synthesis)
- **OCR Calls**: 4 (one per document, with caching)

#### Try It Yourself

After running, you can:
- Inspect `result.document_classifications` to see how documents were classified
- Review `result.invoice_analysis` for detailed invoice breakdowns
- Check `result.compliance_findings` for flagged issues
- Read `result.executive_summary` for the final report

In [ ]:
# Run the workflow on all documents (3 invoices + 1 police report)
document_ids = ["doc_001", "doc_002", "doc_003", "doc_004"]

result = run_document_workflow(document_ids)

---

## Workshop Conclusion

Congratulations! You've completed the Mistral OCR workshop. 🎉

### What You've Learned

1. **Mistral OCR Capabilities**
   - Handwriting and whiteboard recognition
   - Invoice and form processing
   - Mathematical expression extraction
   - Official document handling

2. **Multimodal RAG Pipeline**
   - Combining OCR with LLMs for document understanding
   - Building two-stage processing workflows
   - Leveraging Bedrock's Pixtral Large for analysis

3. **Multi-Agent Systems**
   - Orchestrating specialized agents with Strands
   - Tool-based agent interactions
   - State management in complex workflows
   - Compliance automation and duplicate detection

### Next Steps

**Extend This Workshop:**
- Add new document types (contracts, receipts, medical records)
- Create custom compliance rules for your industry
- Integrate with your document management system
- Build a web interface for document upload and processing

**Production Deployment:**
- Set up auto-scaling for SageMaker endpoints
- Implement batch processing for large document volumes
- Add monitoring and alerting with CloudWatch
- Store results in a database for audit trails

**Learn More:**
- [Mistral OCR Documentation](https://docs.mistral.ai/capabilities/vision/)
- [Strands Agents GitHub](https://github.com/strands-agents/strands-agents)
- [Amazon Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [SageMaker Deployment Best Practices](https://docs.aws.amazon.com/sagemaker/)

### Questions?

Reach out to your workshop facilitator or AWS account team for:
- Production deployment guidance
- Custom use case discussions
- Architecture reviews
- Cost optimization strategies